Libraries

In [1]:

import numpy as np
import pandas as pd

## visualization
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

from plotnine import *
import calmap

import plotly.express as px
import folium

## color pallette
cdr = ['#393e46', '#ff2e63', '#30e3ca'] # grey - red - blue
idr = ['#f8b400', '#ff2e63', '#30e3ca'] # yellow - red - blue

Dataset

In [3]:

## importing datasets
# !ls -lt Coronavirus/dataset/covid_19_clean_complete.csv
full_table = pd.read_csv('dataset/covid_19_clean_complete.csv', parse_dates=['Date'])

## checking for missing value
# full_table.isna().sum()

## dataframe info
# dir(full_table)
# full_table.head()
# full_table.info()


Preprocessing

In [4]:

## filling missing values with NA
full_table.iloc[:,[0]] = full_table[['Province/State']].fillna('NA')

## replacing Mainland china with just China
full_table.iloc[:,[1]] = full_table['Country/Region'].replace('Mainland China', 'China')

full_table[full_table.iloc[:,1] == 'Algeria'].sort_values(by = ['Confirmed', 'Deaths'], ascending = False).head(3)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
7354,NA,Algeria,28.0339,1.6596,2020-03-04,12,0,0
7527,NA,Algeria,28.0339,1.6596,2020-03-05,12,0,0
7181,NA,Algeria,28.0339,1.6596,2020-03-03,5,0,0


In [5]:

## show the latest record with the max cumulative numbers of each colunm and group 
temp = full_table.groupby(['Country/Region', 'Province/State'])['Confirmed', 'Deaths', 'Recovered'].max()

temp.style.background_gradient(cmap = 'Pastel1_r')